In [ ]:
import pandas as pd
import numpy as np
import pandas_ta as ta

In [ ]:
df_strats = pd.read_parquet("btcusd_1-min_data.parquet")
all_ta = list((x for x in dir(pd.DataFrame().ta) if not x.startswith("_") and not x.endswith("_")))
df_strats.Timestamp = pd.to_datetime(df_strats.Timestamp, unit="s")
df_strats.set_index(pd.DatetimeIndex(df_strats["Timestamp"]), inplace=True, drop=True)
if "Timestamp" in df_strats.columns:
    df_strats.drop(columns=["Timestamp"], inplace=True)

In [ ]:
initial_columns = df_strats.columns
dict_added_columns = {}
for strategy in all_ta:
    df_copy = df_strats.head(100).copy(deep=True)
    try:
        strategy_obj = ta.Strategy("a", [{"kind": strategy}])
        df_copy.ta.strategy(strategy_obj)
    except Exception as e:
        print(f"Strategy: {strategy} - Error: {e}")
    added_columns = [x for x in df_copy.columns if x not in initial_columns]
    dict_added_columns[strategy] = added_columns
    print(f"Strategy: {strategy} - Added columns: {len(added_columns)}")

In [ ]:
# Make sure the reason some strategies are not working is not because of how short the data is
for key, value in dict_added_columns.items():
    if len(value) == 0:
        df_copy = df_strats.copy(deep=True)
        try:
            strategy_obj = ta.Strategy("a", [{"kind": key}])
            df_copy.ta.strategy(strategy_obj)
        except Exception as e:
            print(f"Strategy: {key} - Error: {e}")
        added_columns = [x for x in df_copy.columns if x not in initial_columns]
        dict_added_columns[key] = added_columns
        print(f"Strategy: {key} - Added columns: {len(added_columns)}")

In [ ]:
import pickle

# Save dict_added_columns
with open("dict_added_columns.pickle", "wb") as handle:
    pickle.dump(dict_added_columns, handle, protocol=pickle.HIGHEST_PROTOCOL)